In [1]:
import os
import pandas as pd
import numpy as np
import dask.array as da
from tifffile import imread
from scipy.spatial import cKDTree
import napari
import pickle as pkl
from sem_quant.processing_utils import mask_from_df

In [2]:
im_path = r'I:\\CBI\\Jonathan\\CLEM\\Birder\\88EM87 A\\00001_ashlar.ome.tif'
df_path = r'I:\CBI\Jonathan\CLEM\Birder\88EM87 A\88EM87A01_mitos.pkl'
output_dir = r'I:\CBI\Jonathan\CLEM\Birder\88EM87 A\01_annotations'

categories = ['healthy','swollen','discard']
colors = ['green','red','blue']

mitos_res = 1

In [3]:
# read in the image
store = imread(im_path, aszarr=True)
im_list = []
for res in range(3):
    im = da.from_zarr(store,res)
    im_list.append(im)

im_shape = im_list[mitos_res].shape
im_shape

(25535, 32073)

In [4]:
# read in the info about the mitochondria
df_mitos = pd.read_pickle(df_path)

# create the mask
mask = mask_from_df(df_mitos, im_shape)

In [5]:
# visualize image and objects
viewer = napari.Viewer()
viewer.add_image(im_list)
viewer.add_labels(mask, name='mito mask', visible=True, scale = (2**mitos_res, 2**mitos_res))

# add points for each category
for cat, color in zip(categories, colors):

    if cat in df_mitos.columns:
        points = df_mitos.loc[df_mitos[cat] == 1,['centroid-0','centroid-1']].to_numpy()
    else:
        points = []
    
    viewer.add_points(points, name = cat, size = 30, face_color= color, scale = (2**mitos_res, 2**mitos_res))

# SAVE

In [13]:
# Combine all annotation points and keep track of their categories
all_points = []
all_labels = []

for cat in categories:
    points = viewer.layers[cat].data
    all_points.append(points)
    all_labels.extend([cat] * len(points))

# Convert to arrays
all_points = np.vstack(all_points)
all_labels = np.array(all_labels)

# Build KDTree from all points
tree = cKDTree(all_points)

# Query the closest annotation point for each centroid
centroids = df_mitos[['centroid-0', 'centroid-1']].to_numpy()
distances, indices = tree.query(centroids, k=1)

# Apply cutoff and assign annotation if within threshold
cutoff = 20  # distance threshold
df_mitos['annotation'] = [
    all_labels[i] if d <= cutoff else None
    for d, i in zip(distances, indices)
]

# add binary columns for each category
for cat in categories:
    df_mitos[cat] = df_mitos['annotation'] == cat

# save the updated DataFrame to a pickle file
df_mitos.to_pickle(df_path)

# pickle all the points data
for layer in viewer.layers:
    if isinstance(layer, napari.layers.Points):
        filename = f"{layer.name.replace(' ', '_')}.pkl"
        filepath = os.path.join(output_dir, filename)
        data_to_save = {
            'data': layer.data,
            'properties': layer.properties,
            'features': layer.features,
            'metadata': layer.metadata
        }
        with open(filepath, 'wb') as f:
            pkl.dump(data_to_save, f)
        print(f"Saved Points layer '{layer.name}' to {filepath}")

Saved Points layer 'healthy' to I:\CBI\Jonathan\CLEM\Birder\88EM87 A\01_annotations\healthy.pkl
Saved Points layer 'swollen' to I:\CBI\Jonathan\CLEM\Birder\88EM87 A\01_annotations\swollen.pkl
Saved Points layer 'discard' to I:\CBI\Jonathan\CLEM\Birder\88EM87 A\01_annotations\discard.pkl
